<a href="https://colab.research.google.com/github/mertcan-basut/llm-applications/blob/main/llm_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai
!pip install -q python-dotenv

In [94]:
!echo "OPENAI_API_KEY=editme" > .env

In [95]:
from openai import OpenAI as OpenAIClient

from sklearn.model_selection import StratifiedShuffleSplit

import pandas as pd
import numpy as np

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(), override=True) # read local .env file

## Prepare dataset

LLMs' contextual and semantic perception capabilities are exploited for classifying BBC news articles into 5 distinct categories: `tech`, `business`, `sport`, `entertainment`, and `politics`

In [17]:
# Kaggle API Token is downloaded from https://www.kaggle.com/settings
# and uploaded to the file system's working directory

!mkdir .kaggle
!cp kaggle.json .kaggle/
!chmod 600 .kaggle/kaggle.json

!kaggle datasets download yufengdev/bbc-fulltext-and-category
!unzip bbc-fulltext-and-category.zip -d data/
!rm bbc-fulltext-and-category.zip

Dataset URL: https://www.kaggle.com/datasets/yufengdev/bbc-fulltext-and-category
License(s): CC0-1.0
  0% 0.00/1.83M [00:00<?, ?B/s]
100% 1.83M/1.83M [00:00<00:00, 132MB/s]
Archive:  bbc-fulltext-and-category.zip
  inflating: data/bbc-text.csv       


In [73]:
def get_samples(data: pd.DataFrame, categories_col_name: str, samples_len: int):
  """
  Get samples from the dataset while keeping the balance of the samples with respect to the classification categories.

  Args:
    data (pd.DataFrame): The dataset.
    categories_col_name (str): The name of the column containing the classification categories.
    samples_len (int): The number of samples to get.
  """
  categories = data[categories_col_name].unique()
  if samples_len < categories.size:
    categories = np.random.choice(categories, size=samples_len, replace=False)
    samples = pd.concat([data[data[categories_col_name] == category].sample(n=1, random_state=42) for category in categories])
  else:
    for _, test_index in StratifiedShuffleSplit(
        n_splits=1, test_size=samples_len, random_state=42
      ).split(data, data['category']): samples = data.iloc[test_index]
  return samples

In [101]:
data = pd.read_csv("data/bbc-text.csv")
categories = data['category'].unique()

few_shot_samples = get_samples(data, 'category', 5)
test_data = get_samples(data.drop(few_shot_samples.index), 'category', 10)

# Classification

In [96]:
client = OpenAIClient()

## Classification using *function calling*

## Classification using *log probabilities*